## train code 

In [ ]:
# import 

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
from PIL import Image
import random
from matplotlib import pyplot as plt
from torchsummary import summary
import torchvision.transforms as transforms
import time
import torchvision.models as models
from torchsummary import summary as summary_
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
random.seed(777)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## 드라이브

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## image Transform

class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train' : transforms.Compose([
                transforms.Resize((resize, resize)),  # 이미지를 224x224로 조정
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val' : transforms.Compose([
                transforms.Resize((resize, resize)),  # 이미지를 224x224로 조정
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase):
        img = img.convert("RGB")
        return self.data_transform[phase](img)

In [ ]:
# 이미지 불러오기

bag_path = '/content/drive/MyDrive/IMAGE_FIN/MUSINSA_7' # 이미지 주소 

img_list = os.listdir(bag_path)
print(f'가방 개수 :{len(img_list)}')

In [ ]:
## 소분류 개수 주의!!

def ranklist(img_list):        
    rank_0 = []
    rank_1 = []
    rank_2 = []
    rank_3 = []
    rank_else = []
    for img in img_list:
        if img[0] == 'c':
            label = int(img.split('_')[-1][0])
        else:
            name, _ = os.path.splitext(img)
            pr_label = name.split('_')[-1]
            if len(pr_label) == 1:
                label = int(pr_label)
            else:
                label = int(pr_label[1])
            
        if label == 0:
            rank_0.append(img)
        elif label == 1:
            rank_1.append(img)
        elif label == 2:
            rank_2.append(img)
        elif label == 3:
            rank_3.append(img)
        else:
            rank_else.append(img)
    
    return rank_0, rank_1, rank_2, rank_3 ,rank_else


In [ ]:
rank_0, rank_1, rank_2, rank_3, rank_else = ranklist(img_list)

print(f'0등급 개수 : {len(rank_0)}')
print(f'1등급 개수 : {len(rank_1)}')
print(f'2등급 개수 : {len(rank_2)}')
print(f'3등급 개수 : {len(rank_3)}')
print(f'else등급 개수 : {len(rank_else)}')
print(len(rank_0) + len(rank_1) + len(rank_2) + len(rank_3))

In [ ]:
# train, valid 나누기

rank_list = [rank_0, rank_1, rank_2, rank_3]
for index, rl in enumerate(rank_list):
    random.shuffle(rl)
    count = int(len(rl) * 0.8)
    globals()['valid_'+str(index)] = rl[count:]
    globals()['train_'+str(index)] = rl[:count]

In [ ]:
train_img = train_0 + train_1 + train_2 + train_3
valid_img = valid_0 + valid_1 + valid_2 + valid_3

print(f'train 개수 : {len(train_img)}')
print(f'valid 개수 : {len(valid_img)}')

In [ ]:
rank_0, rank_1, rank_2, rank_3, rank_else = ranklist(train_img)

print(f'0등급 개수 : {len(rank_0)}')
print(f'1등급 개수 : {len(rank_1)}')
print(f'2등급 개수 : {len(rank_2)}')
print(f'3등급 개수 : {len(rank_3)}')
print(f'else등급 개수 : {len(rank_else)}')

In [ ]:
rank_0, rank_1, rank_2, rank_3, rank_else = ranklist(valid_img)

print(f'0등급 개수 : {len(rank_0)}')
print(f'1등급 개수 : {len(rank_1)}')
print(f'2등급 개수 : {len(rank_2)}')
print(f'3등급 개수 : {len(rank_3)}')
print(f'else등급 개수 : {len(rank_else)}')

In [ ]:
## 테스트 데이터셋 이미지 확인 함수(소분류 개수에 따라 true_label값 변경, [4=기타 or 2=기타])

def display_image_grid(checkpoint, path, file_name, predicted_labels=(), cols=5): # checkpoint 몇개 확인하고 싶어 # 10의 배수
    rows = checkpoint // cols
    figure, ax = plt.subplots(nrows = rows, ncols = cols, figsize = (12,6))
    random.shuffle(file_name)
    for i, file in enumerate(file_name[:checkpoint]):
        image = cv2.imread(os.path.join(path,file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if file[0] == 'c':
            if file.split('_')[-1][0] == '3':
                true_label = 4
            else:
                true_label = int(file.split('_')[2])
        else:
            if file.split('_')[-1][1] == '3':
                true_label = 4
            else:
                true_label = int(file.split('_')[1])
        predicted_label = predicted_labels[i] if predicted_labels else true_label
        color = 'green' if true_label == predicted_label else 'red'
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_title(predicted_label, color = color)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
display_image_grid(20, bag_path, train_img)

In [ ]:
display_image_grid(20, bag_path, valid_img)

In [ ]:
train= [os.path.join(bag_path, f) for f in train_img]
valid = [os.path.join(bag_path, f) for f in valid_img]

In [ ]:
### 데이터셋 클래스 정의 (소분류 개수에 따라 label값 변경해주기!!)

class rankDataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)
        pre_label = img_path.split('/')[-1]

        if pre_label[0] == 'c':
            if pre_label.split('_')[-1][0] == '3':
                label = 4
            else:
                label = int(pre_label.split('_')[2])
        else:
            if pre_label.split('_')[-1][1] == '3':
                label = 4
            else:
                label = int(pre_label.split('_')[1])
        
        return img_transformed, label

In [ ]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.224)
batch_size = 32

In [ ]:
train_dataset = rankDataset(train, transform=ImageTransform(size, mean, std), phase = 'train')
valid_dataset = rankDataset(valid, transform=ImageTransform(size, mean, std), phase = 'val')

index = 0
print(train_dataset.__getitem__(3)[0].size())
print(train_dataset.__getitem__(3)[1])

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle=False)
dataloader_dict = {'train':train_dataloader , 'val':val_dataloader}

inputs , label = next(iter(train_dataloader))
print(inputs.size())
print(label)

### Model

소분류 개수에 따라 num_classes 값 정의 주의하기!!

In [ ]:
# 전이학습 X

class ResNet50(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=False)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# 전이학습 O

class ResNet50_v1(nn.Module): 
    def __init__(self, num_classes=5):
        super(ResNet50_v1, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = True
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# 입력층에 가까운 5개층 고정

class ResNet50_v2(nn.Module): 
    def __init__(self, num_classes=5):
        super(ResNet50_v2, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        ct = 0
        for child in self.resnet.children():
          ct += 1
          if ct < 6:
            for param in self.resnet.parameters():
              param.requires_grad = False
          else:
            for param in self.resnet.parameters():
              param.requires_grad = True
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# 입력층에 가까운 6개층 고정

class ResNet50_v3(nn.Module): 
    def __init__(self, num_classes=5):
        super(ResNet50_v3, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        ct = 0
        for child in self.resnet.children():
          ct += 1
          if ct < 7:
            for param in self.resnet.parameters():
              param.requires_grad = False
          else:
            for param in self.resnet.parameters():
              param.requires_grad = True
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# layer8, layer10만 학습

class ResNet50_v4(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50_v4, self).__init__()
        self.features = models.resnet50(pretrained=True)
        
        
        # 고정시킬 층의 파라미터를 requires_grad=False로 설정
        for name, param in self.features.named_parameters():
            if "layer8" in name or "layer10" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        num_features = self.features.fc.in_features
        self.features.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        return x

In [ ]:
# layer4에서 conv2층과 fc층 학습

class ResNet50_v5(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50_v5, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        for name, param in self.resnet.named_parameters():
            if ('layer4' in name and 'conv2' in name) or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# layer4에서 bn층과 fc층 학습

class ResNet50_v6(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50_v6, self).__init__()
        self.resnet = models.resnet50(pretrained=True)

        for name, param in self.resnet.named_parameters():
            if ('layer4' in name and 'bn' in name) or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
                
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
# 모델에 따라 이름 변경!!

model_resnet50 = ResNet50()

In [ ]:
optimizer = optim.Adam(model_resnet50.parameters(), lr = 0.003)
criterion = nn.CrossEntropyLoss()

In [ ]:
model_resnet50 = model_resnet50.to(device)
criterion = criterion.to(device)

In [ ]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):    
    best_acc = 0.0 

    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        since = time.time()
        
        for phase in ['train', 'val']:           
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in dataloader_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            writer.add_scalar(f"Loss/{phase}", epoch_loss, epoch)
            writer.add_scalar(f"Acc/{phase}", epoch_acc, epoch)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
            time_elapsed = time.time() - since
            print('Training complete in {:.0f}m {:.0f}s'.format(
                time_elapsed // 60, time_elapsed % 60))
            print('Best val Acc: {:4f}'.format(best_acc))
    writer.flush()
    writer.close()

    return model

In [ ]:
num_epoch = 20
model_resnet50_pre2 = train_model(model_resnet50, dataloader_dict, criterion, optimizer, num_epoch)